# Import Dependencies

In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import time

# Init Splinter Browser

In [3]:
# MAC
# executable_path = { 'executable_path': '/usr/local/bin/chromedriver' }
# WINDOWS
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# Scrape the List of Genre 

In [9]:
# URL to be scraped
url = 'https://www.vgchartz.com/gamedb/?page='

# Open webpage
browser.visit(url)

# Retrieve HTML webpage source
html = browser.html

# Parse HTML webpage source using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Scrape the list of genre
genre_list = []
result_select = soup.find('select', {'name':'genre'})
result_options = result_select.find_all('option')
for result in result_options:
    if result['value'] != '':
        genre_list.append(result['value'])
genre_list

['Action',
 'Action-Adventure',
 'Adventure',
 'Board Game',
 'Education',
 'Fighting',
 'Misc',
 'MMO',
 'Music',
 'Party',
 'Platform',
 'Puzzle',
 'Racing',
 'Role-Playing',
 'Sandbox',
 'Shooter',
 'Simulation',
 'Sports',
 'Strategy',
 'Visual Novel']

# Scrape Games data for each Genre

In [18]:
# Loop and scrape games info for each genre
for genre in genre_list:
    
    # Variable to hold page numbers
    page_num = 1
    
    # Variable to hold if page exists
    page_exist = False
    
    # Build URL to be scraped
    url_base = "https://www.vgchartz.com/games/games.php?"
    url_dyn = f"page={page_num}&results=200&genre={genre.replace(' ', '%20')}"
    url_tail = "&order=Sales&ownership=Both&direction=DESC"
    url_tail += "&showtotalsales=1&shownasales=1&showpalsales=1"
    url_tail += "&showjapansales=1&showothersales=1&showpublisher=1"
    url_tail += "&showdeveloper=1&showreleasedate=1&showlastupdate=1"
    url_tail += "&showvgchartzscore=1&showcriticscore=1&showuserscore=1"
    url_tail += "&showshipped=1"
    
    url = url_base + url_dyn + url_tail
    
    # Open webpage
    browser.visit(url)
    
    # Retrieve HTML webpage source
    html = browser.html
    
    # Parse HTML webpage source using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Logic to check if the page exists
    soup_div = soup.find("div", id="generalBody")
    page_anchors = soup_div.find("tr").find_all("th")[1].find_all("a")
    for a in page_anchors:
        if(a.text.find(str(page_num)) > 0):
            page_exist = True

    while(page_exist):
        url_dyn = f"page={page_num}&results=200&genre={genre.replace(' ', '%20')}"
        url = url_base + url_dyn + url_tail
        
        # Open webpage
        browser.visit(url)
    
        time.sleep(2)
        
        # Retrieve HTML webpage source
        html = browser.html
        
        # Parse HTML webpage source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Scrape the game info into Dataframe
        result = soup.find('div', id='generalBody')
        html_string = result.prettify()
        tbody_start_index = html_string.find("<tbody>\n   <tr>")
        tbody_end_index = html_string.find('<tr>\n    <th style="background-image:url(')
        html_string_start = html_string[0:tbody_start_index + 11]
        html_string_end = html_string[tbody_end_index:]
        new_html_string = html_string_start + html_string_end
        dfs = pd.read_html(new_html_string)
        df = dfs[0]
        
        # Add Genre Column
        df['Genre'] = genre
        
        # Scrape the console info into a list
        console_list = []
        all_trs = result.find('table').find_all('tr')
        tr_count = 0
        for tr in all_trs:
            # Skip first 3 TR tags
            if tr_count > 2:
                console_list.append(tr.find_all("td")[3].find('img').attrs['alt'])
            tr_count += 1;
        print(console_list)
        
        page_exist = False

['Series', 'Series', 'Series', 'Series', 'PS3', 'Series', 'PS4', 'PS2', 'PS4', 'PS2', 'X360', 'PS2', 'PC', 'X360', 'PS4', 'PS3', 'Series', 'PS3', 'XOne', 'PS', 'PSP', 'PS3', 'PC', 'PS2', 'Series', 'PS3', 'PS3', 'X360', 'PS', 'PS', 'PS3', 'PS4', 'Wii', 'PS3', 'X360', 'PS3', 'PC', 'X360', 'PS3', 'PSP', 'PS3', 'PS', 'PS3', 'DS', 'X360', 'PS', 'PS2', 'PS', 'PS2', 'PC', 'All', 'PS2', '3DS', 'PS3', 'PS2', 'PS3', 'X360', 'PC', 'PC', 'PS', 'PS4', 'PS4', 'PS4', 'Series', 'PS4', 'Series', 'PS4', 'PS4', 'All', 'Series', 'PS4', 'PC', 'All', 'PS2', 'PS4', 'XOne', 'PS2', 'PS2', 'X360', 'X360', 'PS2', 'X360', 'PS', 'NES', 'PS', 'PC', 'PS', 'PS2', 'PS4', 'GC', 'PS2', 'PS2', 'PS2', 'PSP', 'Series', 'PS4', 'PS2', 'PS', 'PS3', 'X360', 'WiiU', 'PS4', 'XB', 'All', 'PS3', 'NS', 'All', 'Series', 'PS2', 'All', 'PS4', 'PS3', 'PC', 'XOne', 'Wii', 'X360', 'X360', 'PC', 'PS', 'PS2', 'PS4', 'DS', 'PS2', 'N64', 'GEN', 'X360', 'PC', 'PS3', 'XB', 'PS3', 'PS', 'XOne', 'PC', 'PS4', 'Series', 'PS3', 'PS', 'X360', 'PS2',

['Series', 'Series', 'X360', 'NS', 'Series', 'Series', 'PC', 'Series', 'Series', 'WiiU', 'Series', 'NS', 'Wii', '3DS', 'NS', 'WiiU', 'PC', '3DS', '3DS', 'PS4', 'NS', 'WiiU', '3DS', 'Wii', 'PSP', 'PC', 'PC', 'WiiU', 'PC', 'NS', 'WiiU', 'X360', '3DS', 'PC', '3DS', 'PSP', 'WiiU', 'WiiU', 'PC', 'XB', '3DS', 'PS2', 'NS', 'NS', 'PS4', 'XOne', 'PS4', 'PS3', 'WiiU', 'NS', 'PS4', '3DS', 'PSN', 'PS2', 'PC', 'NS', 'PS4', 'PSN', 'PSP', 'NS', 'NS', 'NS', 'PC', 'XOne', 'PS4', 'PS4', 'NS', 'NS', 'NS', 'X360', 'PS4', 'PS4', 'NS', 'NS', 'WiiU', 'PS4', 'NS', 'WiiU', 'NS', 'PS5', 'XS', 'PS', 'NS', 'PS4', 'NS', 'WiiU', 'PS', 'NS', 'PS4', 'PC', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'PC', 'PS4', 'PS4', 'NS', 'WiiU', 'WiiU']
['Series', 'Series', 'Series', 'Series', 'NES', 'Series', 'Series', 'DS', 'Wii', 'All', 'Series', 'Series', 'SNES', 'Series', 'NS', 'GB', 'NES', 'Series', 'GEN', 'PC', '3DS', 'Wii', '3DS', 'Series', 'N64', 'GB', 'DS', 'SNES', 'All', 'Series', 'SNES', 'NS', 'NES', 'Wii', 'PS', 'Wii', 'All',

['Series', 'Series', 'Series', 'Series', 'Series', 'Wii', 'Series', 'Wii', 'Series', 'All', 'Series', 'Series', 'Wii', 'Series', 'Wii', 'All', 'All', 'Series', 'Series', 'Series', 'All', 'Series', 'All', 'All', 'PS4', 'PS4', 'Series', 'All', 'All', 'All', 'All', 'Series', 'PS4', 'All', 'PS4', 'PS3', 'Series', 'All', 'All', 'All', 'All', 'PS3', 'PS3', 'All', 'PS4', 'Series', 'All', 'PS2', 'X360', 'DS', 'PS3', 'PS', 'All', 'All', 'All', 'Series', 'PS2', 'PS', 'PS3', 'Wii', 'PS2', 'All', 'All', 'All', 'PS2', 'PS2', 'PS2', 'Series', 'X360', 'X360', 'PS2', 'PS2', 'PS2', 'NES', 'All', 'All', 'All', 'PS4', 'Wii', 'PS2', 'PS3', 'PS2', 'XOne', 'PS2', 'PS2', 'PS2', 'PS3', 'PS4', 'X360', 'All', 'All', 'PS2', 'DS', 'PS4', 'PS2', 'Series', 'PS4', 'PS4', 'PS4', 'XOne', 'NES', 'PS2', 'XOne', 'X360', 'PS2', 'NES', 'NS', 'X360', 'All', 'All', 'Series', 'PS2', 'X360', 'X360', 'All', 'X360', 'PS2', 'Wii', 'X360', 'PS2', 'PS', 'PS2', 'PS3', 'X360', 'PS2', 'X360', 'X360', 'X360', 'PS3', 'PS4', 'PS2', 'PS4'